In [2]:
!pip -q install towhee pandas scikit-learn scikit-learn opencv-python ipython matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.8 MB/s eta 0:00:00


There is pipeline for most of the ML activities

Lets see:
pipeline to generate feature vectors for images and normalize them.

Image by <a href="https://pixabay.com/users/hồng_hoàng_sơn-34735808/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=7890229">Hoàng Sơn Hồng</a> from <a href="https://pixabay.com//?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=7890229">Pixabay</a>

Image by <a href="https://pixabay.com/users/betidraws-23986844/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=7904131">bbbeti</a> from <a href="https://pixabay.com//?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=7904131">Pixabay</a>

Image by <a href="https://pixabay.com/users/storme22k-4337202/?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=7913415">Storme22k</a> from <a href="https://pixabay.com//?utm_source=link-attribution&amp;utm_medium=referral&amp;utm_campaign=image&amp;utm_content=7913415">Pixabay</a>

In [1]:
three_img = ["/content/hd-wallpaper-df.jpg",
             "/content/cloud-dawn.jpg",
             "/content/rabbit-pull.jpg"]

In [3]:
import towhee as tw
from towhee import ops

In [4]:
# importing the models and function

image_decode = ops.image_decode()
image_embedding = ops.image_embedding.timm(model_name='resnet50')
tensor_normalize = ops.tensor_normalize()

In [5]:
result = []
for path in three_img:
    img = image_decode(path)
    embedding = image_embedding(img)
    vec = tensor_normalize(embedding)
    result.append(vec)

result

Cloning the repo: towhee/image-decode... Be patient and waiting printing 'Successfully'.
Successfully clone the repo: towhee/image-decode.
Cloning the repo: image-decode/cv2... Be patient and waiting printing 'Successfully'.
Successfully clone the repo: image-decode/cv2.
Cloning the repo: image-embedding/timm... Be patient and waiting printing 'Successfully'.
Successfully clone the repo: image-embedding/timm.


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


[array([0.02448939, 0.00524123, 0.        , ..., 0.00141946, 0.03320975,
        0.        ], dtype=float32),
 array([0.        , 0.        , 0.        , ..., 0.        , 0.00704773,
        0.        ], dtype=float32),
 array([9.6835971e-02, 6.5879125e-05, 3.6558980e-04, ..., 5.9223254e-03,
        5.4870732e-04, 0.0000000e+00], dtype=float32)]

In [ ]:
# As data collection

In [6]:
dc_tw = tw.dc(three_img) \
  .map(ops.image_decode()) \
  .map(ops.image_embedding.timm(model_name='resnet50')) \
  .map(ops.tensor_normalize())

In [ ]:
dc_tw[0]

array([9.6835971e-02, 6.5879125e-05, 3.6558980e-04, ..., 5.9223254e-03,
       5.4870732e-04, 0.0000000e+00], dtype=float32)

In [ ]:
# as dispatcher function

In [7]:
tw.dc(three_img) \
    .image_decode() \
    .image_embedding.timm(model_name='resnet50') \
    .tensor_normalize()

[array([0.0244...dtype=float32), array([0.    ...dtype=float32), array([9.6835...dtype=float32)]

# Tabular data

In [8]:
get_data = tw.read_csv('/content/test.csv') \
    .runas_op['id', 'new_id'](func=lambda x: 'new_'+x) \
    .show()

id,label,score,new_id
0,cat,0.8,new_0
1,dog,0.4,new_1
2,tiger,0.9,new_2


In [9]:
import pandas as pd

df = pd.read_csv('test.csv')
dc_df = tw.from_df(df)
dc_df

   id  label  score
0   0    cat    0.8
1   1    dog    0.4
2   2  tiger    0.9

In [10]:
#towhee has its own dataframe

dc_entity = tw.DataFrame([{'a': i, 
                               'b': i*2} for i in range(5)]).as_entity()
dc_entity.show()

a,b
0,0
1,2
2,4
3,6
4,8


In [11]:
from towhee import register

@register
def mul_5(x):
    return x * 5

tw.ops.mul_5()(2)

10

In [12]:
dc_entity.mul_5['a', 'c']().show()

a,b,c
0,0,0
1,2,5
2,4,10
3,6,15
4,8,20


In [13]:
dc_entity.runas_op['b', 
                   'd'](func=lambda x: x*2).show()

a,b,c,d
0,0,0,0
1,2,5,4
2,4,10,8
3,6,15,12
4,8,20,16


1) Parallel Ops

set_parallel(num_worker=3)

2) Exception handling

exception_safe()


**fill_empty**(default_value) will replace empty values with the default value; 

**drop_empty**(callback=None) will drop the empty values 